In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline

In [2]:
df = pd.read_csv('311_NYPD_FULL.csv')

/Users/TimGimi/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2714: DtypeWarning: Columns (17,34,35,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
keep_cols = ['Created Date', 'Closed Date', 'Complaint Type', 'Descriptor', 'Location Type',
             'Incident Zip', 'Due Date', 'Resolution Description', 'Resolution Action Updated Date',
             'Latitude', 'Longitude','Borough','City']

In [4]:
df = df[keep_cols]
N,C = df.shape
drop_indices = []
for i in range(N):
    if np.sum(pd.isnull(df.iloc[i,:])) != 0:
        drop_indices.append(i)
df = df.drop(df.index[drop_indices])

KeyboardInterrupt: 

Check if we have any missing value left

In [ ]:
np.sum(df.isnull())

Subgroup the following

In [ ]:
comp_type = {'Noise':['Noise - Commercial','Noise - House of Worship','Noise - Park','Noise - Residential','Noise - Street/Sidewalk','Noise - Vehicle'], 
             'Street condition related':['Derelict Vehicle','Graffiti','Homeless Encampment','Panhandling','Vending', 'Posting Advertisement','Sidewalk Condition','Street Condition',],
             'Disturbance':['Bike/Roller/Skate Chronic','Disorderly Youth','Drinking','Drug Activity','Illegal Fireworks','Urinating in Public'],
             'Traffic related': ['Traffic','Illegal Parking','Blocked Driveway'],
             'Miscellaneous': ['Non-Emergency Police Matter','Animal Abuse']}
    

In [ ]:
def set_comp_type(col):
    array = np.zeros(len(col),dtype='object')
    for i,entry in enumerate(col):
        for key,comp_list in comp_type.items():
            if entry in comp_list:
                array[i] = key
    
    return(array)

In [ ]:
df['Complaint Subgroups'] = set_comp_type(df['Complaint Type'])

now we enrich the data, and find the distance to the closest police station.

In [ ]:
df_nypp = pd.read_csv('nypp.csv')

In [ ]:
def get_centroid(col):
    centroid_list = []
    for i,val in enumerate(col):
        v_str = val[val.find('('):].strip('(').strip(')').split(',')
        v_coord = []
        for j,string in enumerate(v_str):
            if string[0] == ' ':
                string = string[1:]
            if string[-1] == ')':
                string = string[:-2]
            if string[0] == '(':
                string = string[2:]
            c0 = float(string[:string.find(' ')].strip(' '))
            c1 = float(string[string.find(' ')+1:].strip(' '))
            if abs(c0+70) >20 or abs(c1-40)>20:
                print('lmao something went south')
            v_coord.append((c0,c1)) 
            
        centroid_list.append(list(np.mean(v_coord,axis = 0).astype('float')))
    return(centroid_list)
def naive_dist(data,prec):
    R = 6371e3
    sqdt = ((np.radians(data)[:,np.newaxis,:]-np.radians(prec)[np.newaxis,:,:])**2)
    naive_geo = np.sqrt(np.sum(sqdt,axis=2))
    return(naive_geo*R)
def calculate_geodesic(data_lonlat,precinct_lonlat):
    """
    lonlat_0 should be the data one, 
    lonlat_1 should be the precinct one
    
      
    """
    R = 6371e3
    data_rad = np.radians(data_lonlat)[:,np.newaxis,:]
    prec_rad = np.radians(precinct_lonlat)[np.newaxis,:,:]
    diff = data_rad-prec_rad
    a = np.sin(diff[:,:,0]/2)**2+np.cos(data_rad[:,:,0])*np.cos(prec_rad[:,:,0])*np.sin(diff[:,:,1]/2)**2
    c = 2*np.arctan2(np.sqrt(a), np.sqrt(1-a))
    
    return c*R

In [ ]:
#%debug
result = get_centroid(df_nypp['the_geom'])
result_arr = np.array(result,dtype='float')

In [ ]:
data_arr = df[['Longitude','Latitude']].values

In [ ]:
#%debug
dist_to_closest_station = np.min(calculate_geodesic(data_arr,result_arr),axis = 1)

In [ ]:
df['Distance To Closest Station'] = dist_to_closest_station

In [ ]:
df.to_csv('cleaned_data_FULL.csv')